### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
purchase_data_df.sort_values('')

## Player Count

* Display the total number of players


In [16]:
#Find total number of players by counting the number of unique screen names.
total_players = purchase_data_df["SN"].nunique()

total_players

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [14]:
#For a top-level summary of in-game purchases, find number of items for sale using nunique(), average price using mean(),
#total purchases using the length of any column in the dataframe, and total revenue using sum().
num_items = purchase_data_df["Item Name"].nunique()
avg_pr = purchase_data_df["Price"].mean()
tot_purchases = len(purchase_data_df["Purchase ID"])
tot_revenue = purchase_data_df["Price"].sum()

#Create a dataframe for the findings
summary_df = pd.DataFrame({
                           "Number of Unique Items": [num_items],
                           "Average Purchase Price": "${0:.2f}".format(avg_pr),
                           "Total Number of Purchases": [tot_purchases],
                           "Total Revenue": "${0:.2f}".format(tot_revenue)
                           })
summary_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [15]:
#For gender demographics, filter the original dataframe by gender. Then use the results to find count and percentage.
male_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Male"]
female_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Female"]
other_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed"]

male_count = male_df["SN"].nunique()
female_count = female_df["SN"].nunique()
other_count = other_df["SN"].nunique()

#Create a dataframe to store the results (percentage of players is calculated  here as well)
gender_demo_df = pd.DataFrame({"Count": [male_count, female_count, other_count],
                               "Percentage of Players": [male_count/total_players, female_count/total_players, other_count/total_players]
                               }, index=["Male", "Female", "Other"])

#Format percentage to look nice
gender_demo_df['Percentage of Players'] = pd.Series(["{0:.2f}%".format(val*100) for val in gender_demo_df['Percentage of Players']], index = gender_demo_df.index)

gender_demo_df

,Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [18]:
#For purchasing analysis by gender, go back to the original dataframe and use groupby() on the gender column.
gender_df = purchase_data_df.groupby(['Gender'])

#Find the first three items of interest using count(), mean(), and sum()
gender_pur_count = gender_df['Purchase ID'].count()
gender_avg_pr = gender_df['Price'].mean()
gender_total_value = gender_df['Price'].sum()

#Begin to create a summary dataframe by merging two of the above dataframes, then inserting Purchase Count
gen_purch_df = pd.merge(gender_avg_pr, gender_total_value, on="Gender")
gen_purch_df = gen_purch_df.rename(columns={"Price_x":"Average Purchase Price","Price_y":"Total Purchase Value"})
gen_purch_df.insert(0,'Purchase Count', gender_pur_count)

#Sort by Total Purchase Value in descending order to make it match the gender demographics dataframe gender_demo_df,
#since that will be used to calculate avg purchase per person.
gen_purch_df = gen_purch_df.sort_values("Total Purchase Value", ascending=False) 

#Add a column for average per person and do the calculation
gen_purch_df['Avg Total Purchase per Person'] = round(gen_purch_df['Total Purchase Value'] / list(gender_demo_df['Count']),2)

#Format
gen_purch_df['Average Purchase Price'] = pd.Series(["${0:.2f}".format(val) for val in gen_purch_df['Average Purchase Price']], index = gen_purch_df.index)
gen_purch_df['Total Purchase Value'] = pd.Series(["${0:.2f}".format(val) for val in gen_purch_df['Total Purchase Value']], index = gen_purch_df.index)
gen_purch_df['Avg Total Purchase per Person'] = pd.Series(["${0:.2f}".format(val) for val in gen_purch_df['Avg Total Purchase per Person']], index = gen_purch_df.index)

gen_purch_df



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,$1967.64,$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#Define bins for age groups and add a column 'Age Group' to the original dataframe to hold this info.
bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]
group_names = ['< 10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

purchase_data_df['Age Group'] = pd.cut(purchase_data_df["Age"], bins, labels=group_names, include_lowest=True)

#groupby() the new Age Group column, create a summary dataframe, and format percentages.
age_df = purchase_data_df.groupby('Age Group')

age_summary_df = pd.DataFrame({'Total Count': age_df['SN'].nunique(),
                              'Percentage of Players': age_df['SN'].nunique()/total_players})

age_summary_df['Percentage of Players'] = pd.Series(["{0:.2f}%".format(val*100) for val in age_summary_df['Percentage of Players']], index = age_summary_df.index)

age_summary_df

,Total Count,Percentage of Players
Age Group,,
< 10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [21]:
#Original purchase data dataframe is already binned and grouped by ages in age_df. 
#Begin creating a purchasing analysis by age summary dataframe by using count, mean, and sum on that grouped object. 
age_purchases_df = pd.DataFrame({'Purchase Count': age_df["Age Group"].count(),
                                 'Average Purchase Price': age_df["Price"].mean(),
                                 'Total Purchase Value': age_df["Price"].sum()
                                })

#Find average per person using total purchase value from the dataframe just created 
#and count by age group from the age demographics summary, and add it to the purchasing summary.
age_purchases_df['Average Total Purchase per Person'] = age_purchases_df['Total Purchase Value'] / list(age_summary_df['Total Count'])

age_purchases_df['Average Purchase Price'] = pd.Series(["${0:.2f}".format(val) for val in age_purchases_df['Average Purchase Price']], index = age_purchases_df.index)
age_purchases_df['Total Purchase Value'] = pd.Series(["${0:.2f}".format(val) for val in age_purchases_df['Total Purchase Value']], index = age_purchases_df.index)
age_purchases_df['Average Total Purchase per Person'] = pd.Series(["${0:.2f}".format(val) for val in age_purchases_df['Average Total Purchase per Person']], index = age_purchases_df.index)

age_purchases_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Group,,,,
< 10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Now looking at spending patterns of individual players, go back to the original dataframe, group by SN, and perform
#similar operations to previous steps.  Sort in descending order to identify top spenders.
spenders = purchase_data_df.groupby("SN")
top_spenders_df = pd.DataFrame({'Purchase Count': spenders["Purchase ID"].count(),
                                 'Average Purchase Price': spenders["Price"].mean(),
                                 'Total Purchase Value': spenders["Price"].sum()
                                })
top_spenders_df = top_spenders_df.sort_values("Total Purchase Value", ascending=False)

top_spenders_df['Average Purchase Price'] = pd.Series(["${0:.2f}".format(val) for val in top_spenders_df['Average Purchase Price']], index = top_spenders_df.index)
top_spenders_df['Total Purchase Value'] = pd.Series(["${0:.2f}".format(val) for val in top_spenders_df['Total Purchase Value']], index = top_spenders_df.index)

top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [28]:
#Now looking at items for purchase, again similar operations to previous analyses. Only difference here is groupby() on two columns from
#the original dataframe, since we want to see item ID and Item Name.

items = purchase_data_df.groupby(["Item ID", "Item Name"])
items_df = pd.DataFrame({'Purchase Count': items["Purchase ID"].count(),
                         'Average Purchase Price': items["Price"].mean(),
                         'Total Purchase Value': items["Price"].sum(),
                         })
items_df = items_df.sort_values("Purchase Count", ascending=False)

items_df['Average Purchase Price'] = pd.Series(["${0:.2f}".format(val) for val in items_df['Average Purchase Price']], index = items_df.index)
items_df['Total Purchase Value'] = pd.Series(["${0:.2f}".format(val) for val in items_df['Total Purchase Value']], index = items_df.index)

items_df.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
items_df = items_df.sort_values("Total Purchase Value", ascending=False)
items_df.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
143,Frenzied Scimitar,6,$1.56,$9.36
